# Preporcessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import sqlite3 as sql
import pandas as pd
import numpy as np

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler

## Database Connection

In [ ]:
con = sql.connect('/content/drive/MyDrive/NN_Praktikum/data/database.sqlite') # need to change path to database

In [ ]:
tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", con)
tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


In [ ]:
player_attributes = pd.read_sql("SELECT * FROM Player_Attributes;", con)
player = pd.read_sql("SELECT * FROM Player;", con)
matches = pd.read_sql("SELECT * FROM Match;", con)
league = pd.read_sql("SELECT * FROM League;", con)
country = pd.read_sql("SELECT * FROM Country;", con)
team = pd.read_sql("SELECT * FROM Team;", con)
team_attributes = pd.read_sql("SELECT * FROM Team_Attributes;", con)

In [ ]:
matches = matches.sort_values(by = 'date', ascending = True)

## Data Analysis

### Scaler

In [ ]:
std_scaler = StandardScaler()

In [ ]:
minmax_scaler = MinMaxScaler() 

## Match Labels

In [ ]:
def get_match_label(match):
    home_goals = match['home_team_goal']
    away_goals = match['away_team_goal']
     
    label = pd.Series(0, index=['Home_Win', 'Draw', 'Away_Win']) 

    if home_goals > away_goals:
        label.loc['Home_Win'] = 1
    if home_goals == away_goals:
        label.loc['Draw'] = 1
    if home_goals < away_goals:
        label.loc['Away_Win'] = 1
        
    return label.to_frame().T

## Fifa Statistics

In [ ]:
home_players = ['home_player_1', 'home_player_2', 'home_player_3', 'home_player_4', 
                'home_player_5', 'home_player_6', 'home_player_7', 'home_player_8', 
                'home_player_9', 'home_player_10','home_player_11']
away_players = ['away_player_1', 'away_player_2', 'away_player_3', 'away_player_4',
                'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8', 
                'away_player_9', 'away_player_10', 'away_player_11']

Aggregation of the fifa statistics for a given match:

In [ ]:
def get_fifa_statistics(match, player_attributes):
    match_id =  match['match_api_id']
    date = match['date']
    home_player_curr_stats = pd.DataFrame()
    away_player_curr_stats = pd.DataFrame()
    home_overall_rating = pd.DataFrame()
    away_overall_rating = pd.DataFrame()
    overall_rating = pd.DataFrame()
    
    for home_player, away_player in zip(home_players, away_players):
        home_player_id = match[home_player]
        away_player_id = match[away_player]
        
        if np.isnan(home_player_id) == True:
            home_player_curr_stats['overall_rating'] = pd.Series(0)
        else:
            home_palyer_stats = player_attributes[player_attributes.player_api_id == home_player_id]
            home_player_curr_stats = home_palyer_stats[home_palyer_stats.date < date].sort_values(by = 'date', ascending = False)[:1]
            home_player_curr_stats.reset_index(inplace = True, drop = True)
        
        if np.isnan(away_player_id) == True:
            away_player_curr_stats['overall_rating'] = pd.Series(0)
        else:
            away_palyer_stats = player_attributes[player_attributes.player_api_id == away_player_id]
            away_player_curr_stats = away_palyer_stats[away_palyer_stats.date < date].sort_values(by = 'date', ascending = False)[:1]
            away_player_curr_stats.reset_index(inplace = True, drop = True)
        
        home_overall_rating = pd.concat([home_overall_rating, pd.Series(home_player_curr_stats['overall_rating'])], axis=1)
        away_overall_rating = pd.concat([away_overall_rating, pd.Series(away_player_curr_stats['overall_rating'])], axis=1)
    
    overall_rating['home_overall_rating'] = home_overall_rating.sum(axis=1)
    overall_rating['away_overall_rating'] = away_overall_rating.sum(axis=1)
    
    return overall_rating

## Last Matches

Get last n matches of a given team:

In [ ]:
def get_last_matches(matches, date, team, n=5):
    team_matches = matches[(matches['home_team_api_id'] == team) | 
                           (matches['away_team_api_id'] == team)]
    last_matches = team_matches[team_matches.date < date].sort_values(by = 'date', ascending = False).iloc[0:n,:]
    return last_matches

Get last n matches __against each other__:

In [ ]:
def get_last_matches_against_eachother(matches, date, home_team, away_team, n=5):
    home_matches = matches[(matches['home_team_api_id'] == home_team) & (matches['away_team_api_id'] == away_team)]    
    away_matches = matches[(matches['home_team_api_id'] == away_team) & (matches['away_team_api_id'] == home_team)]  
    all_matches = pd.concat([home_matches, away_matches])
    
    try:    
        last_matches = all_matches[all_matches.date < date].sort_values(by = 'date', ascending = False).iloc[0:n,:]
    except:
        last_matches = all_matches[all_matches.date < date].sort_values(by = 'date', ascending = False).iloc[0:all_matches.shape[0],:]
    
    return last_matches

## Wins, Draws, Defeats and Goals

Get the number of __wins__ of a given team from a set of matches:

In [ ]:
def get_wins(matches, team, info:str):
    home_wins = int(matches.home_team_goal[(matches.home_team_api_id == team) & (matches.home_team_goal > matches.away_team_goal)].count())
    away_wins = int(matches.away_team_goal[(matches.away_team_api_id == team) & (matches.away_team_goal > matches.home_team_goal)].count())

    total_wins = home_wins + away_wins

    return pd.Series(total_wins, name=info)     

Get the number of __draws__ of a given team from a set of matches:

In [ ]:
def get_draws(matches, team, info:str):
    home_draws = int(matches.home_team_goal[(matches.home_team_api_id == team) & (matches.home_team_goal == matches.away_team_goal)].count())
    away_draws = int(matches.away_team_goal[(matches.away_team_api_id == team) & (matches.away_team_goal == matches.home_team_goal)].count())

    total_draws = home_draws + away_draws

    return pd.Series(total_draws, name=info)   

Get the number of __defeats__ of a given team from a set of matches:

In [ ]:
def get_defeats(matches, team, info:str):
    home_defeats = int(matches.home_team_goal[(matches.home_team_api_id == team) & (matches.home_team_goal < matches.away_team_goal)].count())
    away_defeats = int(matches.away_team_goal[(matches.away_team_api_id == team) & (matches.away_team_goal < matches.home_team_goal)].count())

    total_defeats = home_defeats + away_defeats

    return pd.Series(total_defeats, name=info)     

Get the __goals difference__ of a given team from a set of matches:

In [ ]:
def get_goals_diff(matches, team, info:str):
    home_goals = int(matches.home_team_goal[matches.home_team_api_id == team].sum())
    away_goals = int(matches.away_team_goal[matches.away_team_api_id == team].sum())

    total_goals = home_goals + away_goals
    
    home_goals_rec = int(matches.home_team_goal[matches.away_team_api_id == team].sum())
    away_goals_rec = int(matches.away_team_goal[matches.home_team_api_id == team].sum())
    
    total_goals -= (home_goals_rec + away_goals_rec)
    
    return pd.Series(total_goals, name=info)

## Bet Statistics

In [ ]:
def get_bet_statistics(match):
    bet_statistics_home_cat = ['B365H', 'BWH', 'IWH', 'LBH', 'PSH', 'WHH', 'SJH', 'VCH', 'GBH', 'BSH']
    bet_statistics_draw_cat = ['B365D', 'BWD', 'IWD', 'LBD', 'PSD', 'WHD', 'SJD', 'VCD', 'GBD', 'BSD']
    bet_statistics_away_cat = ['B365A', 'BWA', 'IWA', 'LBA', 'PSA', 'WHA', 'SJA', 'VCA', 'GBA', 'BSA']
    
    bet_statistics_home = match[bet_statistics_home_cat]
    bet_statistics_draw = match[bet_statistics_draw_cat]
    bet_statistics_away = match[bet_statistics_away_cat]
    
    bet_statistics_home.fillna(0, inplace=True)
    bet_statistics_draw.fillna(0, inplace=True)
    bet_statistics_away.fillna(0, inplace=True)
    
    bet_statistics_home = pd.Series(bet_statistics_home.sum(), name='bet_statistics_home')
    bet_statistics_draw = pd.Series(bet_statistics_draw.sum(), name='bet_statistics_draw')
    bet_statistics_away = pd.Series(bet_statistics_away.sum(), name='bet_statistics_away')
    
    return pd.concat([bet_statistics_home, bet_statistics_draw, bet_statistics_away], axis=1)

## Match Statistics

In [ ]:
def get_match_statistics(match, player_attributes):
    # home and away team statistics
    teams_statistics = get_fifa_statistics(match, player_attributes)
    
    # match bet statistics
    match_bet_statistics = get_bet_statistics(match)
    
    # match label
    label = get_match_label(match)

    match_statistics = pd.DataFrame()

    home_team_api_id = match['home_team_api_id']
    away_team_api_id = match['away_team_api_id']
    date = match['date']

    match_statistics.insert(0, 'home_team_api_id', [home_team_api_id], True) 
    match_statistics.insert(1, 'away_team_api_id', [away_team_api_id], True)
    match_statistics.insert(2, 'date', [date], True) 
    
    match_statistics = pd.concat([match_statistics,
                                  teams_statistics,
                                  match_bet_statistics,
                                  label],
                                  axis=1)
    return match_statistics

In [ ]:
def get_match_statistics_with_last_games(match, matches, player_attributes, n=5):
    match_id = match['match_api_id']
    date = match['date']
    home_team = match['home_team_api_id']
    away_team = match['away_team_api_id']
    
    # home and away team statistics
    teams_statistics = get_fifa_statistics(match, player_attributes)
    
    # match statisitcs of home team n last games
    last_matches_home = get_last_matches(matches, date, home_team, n)
    
    home_team_wins = get_wins(last_matches_home, home_team, 'home_team_wins')
    home_team_draws = get_draws(last_matches_home, home_team, 'home_team_draws')
    home_team_defeats = get_defeats(last_matches_home, home_team, 'home_team_defeats')
    home_team_goals_diff = get_goals_diff(last_matches_home, home_team, 'home_team_goals_diff')
    
    # match statisitcs of away team n last games
    last_matches_away = get_last_matches(matches, date, away_team, n)
    
    away_team_wins = get_wins(last_matches_away, away_team, 'away_team_wins')
    away_team_draws = get_draws(last_matches_away, away_team, 'away_team_draws')
    away_team_defeats = get_defeats(last_matches_away, away_team, 'away_team_defeats')
    away_team_goals_diff = get_goals_diff(last_matches_away, away_team, 'away_team_goals_diff')
    
    # match statistics of last n matches against eachother 
    last_matches_against_eachother = get_last_matches_against_eachother(matches, date, home_team, away_team, n)
    
    home_team_wins_against_eachother = get_wins(last_matches_against_eachother, home_team , 'home_team_wins_against_eachother')
    away_team_wins_against_eachother = get_wins(last_matches_against_eachother, away_team, 'away_team_wins_against_eachother')
    draws_against_eachother = get_draws(last_matches_against_eachother, home_team, 'draws_against_eachother')
    
    home_team_goals_diff_agains_eachother = get_goals_diff(last_matches_against_eachother, home_team, 'home_team_goals_diff_agains_eachother')
    away_team_goals_diff_agains_eachother = get_goals_diff(last_matches_against_eachother, away_team, 'away_team_goals_diff_agains_eachother')
    
    # match bet statistics
    match_bet_statistics = get_bet_statistics(match)
    
    # match label
    label = get_match_label(match)
    
    match_statistics_with_last_games = pd.concat([teams_statistics,  
                                  home_team_wins, home_team_draws, home_team_defeats, home_team_goals_diff,
                                  away_team_wins, away_team_draws, away_team_defeats, away_team_goals_diff,
                                  home_team_wins_against_eachother, home_team_goals_diff_agains_eachother,
                                  draws_against_eachother,
                                  away_team_wins_against_eachother, away_team_goals_diff_agains_eachother,
                                  match_bet_statistics,
                                  label],
                                  axis=1)
    return match_statistics_with_last_games

In [ ]:
def get_match_statistics_against_eachother(match, player_attributes):
    # home and away team statistics
    teams_statistics = get_fifa_statistics(match, player_attributes)
    
    # match bet statistics
    match_bet_statistics = get_bet_statistics(match)
    
    # match label
    label = get_match_label(match)
    
    match_statistics = pd.concat([teams_statistics,
                                  match_bet_statistics,
                                  label],
                                  axis=1)
    return match_statistics

In [ ]:
def get_match_statistics_last_games_against_eachother(matches, date, home_team, away_team, player_attributes, n=5):
  columns = ['home_overall_rating', 'away_overall_rating', 'bet_statistics_home', 'bet_statistics_draw', 'bet_statistics_away', 'Home_Win', 'Draw', 'Away_Win']
  columns_rev = ['away_overall_rating', 'home_overall_rating', 'bet_statistics_away', 'bet_statistics_draw', 'bet_statistics_home', 'Away_Win', 'Draw', 'Home_Win']
  label_columns = ['Home_Win', 'Draw', 'Away_Win']
  categories = ['home_team_api_id', 'away_team_api_id', 'date']
  last_matches_against_eachother = get_last_matches_against_eachother(matches, date, home_team, away_team, n) # get last matches against eachother
  match_statistics_against_eachother = pd.DataFrame()
  for i in range(len(last_matches_against_eachother)):
    match = last_matches_against_eachother.iloc[i]
    if match['home_team_api_id'] != home_team: 
      match[columns] = match[columns_rev]
    match.drop(categories, inplace=True)
    match_statistics_against_eachother = match_statistics_against_eachother.append(match)
  diff = n - len(match_statistics_against_eachother) 
  if diff != 0: # if there are not as much last games against eachother than the sequence length requires, generate zero filled data
    for i in range(diff):
      match_statistics_against_eachother = match_statistics_against_eachother.append(pd.Series(0, index=columns), ignore_index=True)
  match_statistics_against_eachother = match_statistics_against_eachother.drop(label_columns, axis=1).iloc[::-1] # drop label
  return match_statistics_against_eachother

### Scaling

In [ ]:
def std_scaling(table, categories=None):
    table_categories = table.loc[:,categories]
    table = table.drop(categories, axis=1) # remove categories/labels during scaling
    table_columns = (table.columns).tolist() 
    scaled_table = std_scaler.fit_transform(table) # standard scaling
    scaled_frame = pd.DataFrame(scaled_table, columns=table_columns)
    table_categories.reset_index(drop=True, inplace=True)
    scaled_frame.reset_index(drop=True, inplace=True)
    return pd.concat([scaled_frame, table_categories], axis=1) # add labels to the scaled data

In [ ]:
def minmax_scaling(table, categories=None):
    table_categories = table.loc[:,categories] 
    table = table.drop(categories, axis=1) # remove categories/labels during scaling
    table_columns = (table.columns).tolist()
    scaled_table = minmax_scaler.fit_transform(table) # minmax scaling
    scaled_frame = pd.DataFrame(scaled_table, columns=table_columns)
    table_categories.reset_index(drop=True, inplace=True)
    scaled_frame.reset_index(drop=True, inplace=True)
    return pd.concat([scaled_frame, table_categories], axis=1) # add labels to the scaled data

## Apply for all Matches

In [ ]:
categories = ['home_team_api_id', 'away_team_api_id', 'date', 'Home_Win', 'Draw', 'Away_Win']

In [ ]:
match_statistics = pd.DataFrame()
r = 1
for row in range(len(matches)):
  print('\r', ' Match: '+str(r)+'/'+str(len(matches)), end='')
  match_statistics = match_statistics.append(get_match_statistics(matches.iloc[row], player_attributes))
  r+=1

In [ ]:
match_statistics_minmax_scaled = minmax_scaling(match_statistics, categories)

Including data from last games:

In [ ]:
match_statistics_with_last_games = pd.DataFrame()
for row in range(len(matches)):
  match_statistics_with_last_games = match_statistics_with_last_games.append(get_match_statistics_with_last_games(matches.iloc[row], matches, player_attributes))

In [ ]:
match_statistics_with_last_games_minmax_scaled = minmax_scaling(match_statistics_with_last_games, labels)

## Save Results

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
match_statistics.to_csv('match_statistics_seq.csv') # need to change name for different datasets
match_statistics_minmax_scaled.to_csv('match_statistics_seq_minmax_scaled.csv') # need to change name for different scaled datasets

In [ ]:
!cp match_statistics_seq.csv "drive/My Drive/"  # need to change name for different datasets
!cp match_statistics_seq_minmax_scaled.csv "drive/My Drive/" # need to change name for different scaled datasets

## Model Training Data

In [ ]:
import torch

In [ ]:
kfolds = 5                          # Cross-Validation folds
batch_size = len(matches) // kfolds # Batch-Size per cross-validation fold
validation_data_percentage = 0.2    # Percentage of validation data per fold

In [ ]:
def batches_(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [ ]:
def get_batches(matches, batch_size):
  batches = []
  for batch in batches_(matches, batch_size):
    if len(batch) != batch_size:
      continue
    batches.append(batch)
  return batches

## Proecss MLP Training Data for Block Cross-Validation

In [ ]:
  label_columns = ['Home_Win', 'Draw', 'Away_Win']

Without last matches information:

In [ ]:
match_statistics = pd.read_csv("/content/drive/MyDrive/NN_Praktikum/data/match_statistics_minmax_scaled.csv", index_col=[0])

In [ ]:
match_statistics_label_tensor = torch.tensor((match_statistics[label_columns].values).astype(np.float32))
match_statistics.drop(label_columns, axis=1, inplace=True)
match_statistics_tensor = torch.tensor((match_statistics.values).astype(np.float32))

In [ ]:
training_data_batches = get_batches(match_statistics_tensor, batch_size)
training_data_lables_batches = get_batches(match_statistics_label_tensor, batch_size)
training_data_batches = [training_data_batches, training_data_lables_batches]

In [ ]:
torch.save(training_data_batches, '/content/drive/MyDrive/NN_Praktikum/training_data/training_data_batches.pt')

With last matches information:

In [ ]:
match_statistics_with_last_games = pd.read_csv("/content/drive/MyDrive/NN_Praktikum/data/match_statistics_with_last_games_minmax_scaled.csv", index_col=[0])

In [ ]:
match_statistics_with_last_games_label_tensor = torch.tensor((match_statistics_with_last_games[label_columns].values).astype(np.float32))
match_statistics_with_last_games.drop(label_columns, axis=1, inplace=True)
match_statistics_with_last_games_tensor = torch.tensor((match_statistics_with_last_games.values).astype(np.float32))

In [ ]:
training_data_batches = get_batches(match_statistics_with_last_games_tensor, batch_size)
training_data_lables_batches = get_batches(match_statistics_with_last_games_label_tensor, batch_size)
training_data_batches = [training_data_batches, training_data_lables_batches]

In [ ]:
torch.save(training_data_batches, '/content/drive/MyDrive/NN_Praktikum/training_data/training_data_with_last_games_batches.pt')

## Process LSTM Training Data for Blocked Cross-Validation

In [ ]:
match_statistics_seq = pd.read_csv("/content/drive/MyDrive/NN_Praktikum/data/match_statistics_seq_minmax_scaled.csv", index_col=[0])

In [ ]:
seq_length = 5

In [ ]:
# Generates sequential data for LSTM
def get_training_data_batches(matches, batches, player_attributes):
  training_data_batches = []
  i = 1
  for batch in batches:
    training_data = []
    training_data_labels = []
    m = 1
    for _, match in batch.iterrows():
      print('\r', 'Batch: '+str(i)+'/'+str(len(batches))+' Match: '+str(m)+'/'+str(len(batch)), end='')
      # match_statistics = get_match_statistics(match, player_attributes)
      date = match['date']
      home_team_api_id = match['home_team_api_id']
      away_team_api_id = match['away_team_api_id']
      # get label
      label = match[['Home_Win', 'Draw', 'Away_Win']]
      # get sequence of last games against each other
      match_statistics_last_games_against_eachother = get_match_statistics_last_games_against_eachother(matches, date, home_team_api_id, away_team_api_id, player_attributes, n=seq_length)
      # concatenate to sequence
      match.drop(['home_team_api_id', 'away_team_api_id', 'date', 'Home_Win', 'Draw', 'Away_Win'], inplace=True)
      match = pd.concat([match_statistics_last_games_against_eachother, match.to_frame().T], axis=0)
      match_statistics_tensor = torch.tensor((match.values).astype(np.float32))
      training_data.append(match_statistics_tensor)
      label_tensor = torch.tensor((label.values).astype(np.float32))
      training_data_labels.append(label_tensor)
      m += 1
    training_data_tensor = torch.stack(training_data)
    training_data_labels_tensor = torch.squeeze(torch.stack(training_data_labels))
    training_data_batches.append([training_data, training_data_labels])
    i += 1
  return training_data_batches

In [ ]:
batches = get_batches(match_statistics_seq, batch_size)
training_data_batches = get_training_data_batches(match_statistics_seq, batches, player_attributes)

 Batch: 5/5 Match: 5195/5195

In [ ]:
torch.save(training_data_batches, '/content/drive/MyDrive/NN_Praktikum/training_data/training_data_seq_batches.pt')